# Esercitazione MLPClassfier con GridSearch

## Indice dei contenuti

- [Import delle librerie](#Import-delle-librerie)
- [Caricamento in memoria del dataset](#Caricamento-in-memoria-del-dataset)
- [Analisi Esplorativa](#Analisi-Esplorativa)
- [Preprocessing](#Preprocessing)
    - [Z-Normalization](#Z-Normalization)
    - [Trasformazione della feature target da categorico a numerico](#Trasformazione-della-feature-target-da-categorico-a-numerico)
    - [Creazione del nuovo dataframe](#Creazione-del-nuovo-dataframe)
    - [Split dei dati in train e test set](#Split-dei-dati-in-train-e-test-set)
- [GridSearch e Funzioni di Attivazione](#GridSearch-e-Funzioni-di-Attivazione)
- [Valutazione](#Valutazione)

## Import delle librerie

In [ ]:
#Import delle librerie
import numpy as np
import pandas as pd 
import seaborn as sns 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

## Caricamento in memoria del dataset

In [ ]:
col_names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm', 'Species']
data = pd.read_csv("../input/iris-flower-dataset/IRIS.csv", names = col_names, header=0) 
data.sample(5) #Stampa di alcuni elementi del dataset

In [ ]:
data.info()

## Analisi Esplorativa

In [ ]:
import seaborn as sns
sns.pairplot( data=data, vars=('SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm'), hue='Species' )

In [ ]:
data.describe()

## Preprocessing

### Z Normalization

La standardizzazione è un procedimento che riconduce una variabile aleatoria distribuita secondo una media μ e varianza σ2, ad una variabile aleatoria con distribuzione "standard", ossia di media zero e varianza pari a 1. È particolarmente utile nel caso della variabile casuale normale per il calcolo della funzione di ripartizione e dei quantili con le tavole della normale standard. Infatti i valori della distribuzione normale sono tabulati per media zero e varianza unitaria.

Il procedimento prevede di sottrarre alla variabile aleatoria la sua media e dividere il tutto per la deviazione standard (per σ e non per σ2), ovvero utilizzando la formula utile a trovare i punti zeta (Z-score o standard score):

<img align="center" src="https://d1whtlypfis84e.cloudfront.net/guides/wp-content/uploads/2020/04/04155631/1426878678.png"/>

In [ ]:
df_norm = data[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df_norm.sample(n=5)

In [ ]:
df_norm.describe()

### Trasformazione della feature target da categorico a numerico

In [ ]:
target = data[['Species']].replace(['Iris-setosa','Iris-versicolor','Iris-virginica'],[0,1,2])
target.sample(n=5)

### Creazione del nuovo dataframe

In [ ]:
df = pd.concat([df_norm, target], axis=1)
df.sample(n=5)

### Split dei dati in train e test set

In [ ]:
train, test = train_test_split(df, test_size = 0.3)
X_train = train[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
y_train = train.Species
X_test = test[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
y_test = test.Species

## GridSearch e Funzioni di Attivazione

Di seguito vengono riportate le possibili funzioni di attivazione utilizzate per l'aggiornamento dei pesi dei neuroni.
Tali funzioni vengono valutate mediante una GridSearch, in modo da individuare la combinazione migliore di parametri.


1. <b>relu</b>: La Rectifier Function è la funzione di attivazioni più utilizzata. Restituisce 0 qualora la somma pesata dei segnali in input è minore o uguale a zero, oppure ΣwX negli altri casi. Il codominio della funzione spazia in questo caso da 0 ad infinito.
2. <b>selu</b>: E' una variante della relu. Per valori positivi, restituisce ΣwX, per valori negativi, l'andamento della funzione ricorda il grafico della funzione logaritmo. Può essere utilizzata quando la funzione relu crea il problema noto come "dying relu", ovvero quando tutti gli output assumono tutti lo stesso valore. Questo accade quando si è in presenza di valori molto piccoli, quindi prossimi allo zero. Dato che il gradiente di zero è zero, la rete neurale non è in grado di aggiornare i pesi.
3. <b>tanh</b>: si comporta come la sigmoide, ma il range di valori è [-1, 1]. Il vantaggio è che gli input saranno mappati fortemente in modo negativo, e i valori prossimi allo zero saranno mappati come zero in tanh.
5. <b>sigmoid</b>: in questo caso il codominio della funzione, ovvero i valori che può restituire il neurone, spazia tra 0 ed 1 in un intervallo continuo. Infatti, la caratteristica di questa funzione è che smussata. Può essere utilizzata al posto della Threshold Function considerando il valore in uscita non come Y ma come probabilità che Y sia uguale ad uno, ovvero Prob(Y=1).

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

grid = {'solver': ['lbfgs', 'sgd', 'adam'], 'activation': ['identity', 'logistic', 'tanh', 'relu']}
clf_cv = GridSearchCV(MLPClassifier(random_state=1, max_iter=5000, hidden_layer_sizes=(3,3), alpha=1e-5), grid, n_jobs=-1, cv=10)

clf_cv.fit(X_train, y_train)

print("GridSearch():\n")
combinazioni = 1
for x in grid.values():
    combinazioni *= len(x)
print('Per l\'applicazione della GridSearch ci sono {} combinazioni'.format(combinazioni))
print("Migliore configurazione: ",clf_cv.best_params_)
best_config_gs = clf_cv.best_params_
print("Accuracy CV:",clf_cv.best_score_)
ppn_cv = clf_cv.best_estimator_
print('Test accuracy: %.3f' % clf_cv.score(X_test, y_test))


Nello snippet seguente viene creato un MLPClassifier che abbia come parametri queli del miglior modello individuato dalla GridSearch.

In [ ]:
mlp = MLPClassifier(random_state=1, max_iter=5000, hidden_layer_sizes=(3,3), alpha=1e-5, **best_config_gs)

mlp.fit(X_train,y_train)
predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

## Valutazione

Successivamente viene creata una matrice di confusione e un report di classificazione per il Train Set, individuando una serie di misure utili alla valutazione.

In [ ]:
#Matrice di confusione e report di classificazione per il Train
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

Infine la stessa matrice e lo stesso report vengono creati per il Test Set.

In [ ]:
#Matrice di confusione e report di classificazione per il Test
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))